# 1. Library

In [1]:
import os
import torch

import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import sklearn.preprocessing
#from directory_tree import display_tree
# Customed Library
import engine ,model_builder,utils
import data_loader
import numpy as np
#import torchinfo
from timeit import default_timer as timer 
from ros_data_reader import Vector_set
#display_tree('./')
import matplotlib.pyplot as plt

# 2. Data loading

In [3]:
path='./first_trial'
Desired_param=[[0],[0]]
New_Columns=['Par1','Par2']

A=data_loader.data_loader(path,Desired_param)
A.file_searching(path)

1st Folder name :  first_0
Contents in the folder : ['metadata.yaml', 'first_0.db3'] 


1 Parameters ['first'] 

2 Parameters ['0'] 



In [4]:
#path='./rosbag_dir'
#Desired_param=[[0],[0],[0],[0],[0,1,2]]
#New_Columns=['Par1','Par2','Par3','Par4','Par5']

path='./first_trial'
Desired_param=[[0],[0]]
New_Columns=['obj_p1','obj_p2']

A=data_loader.data_loader(path,Desired_param)
A.file_searching(path)
list_file=A.desired_file_path(Desired_param)
Columns=['Force_X','Force_Y','Force_Z','Torque_X','Torque_Y','Torque_Z',
         'Input_P_1','Input_P_2','Flag',
         'TCP_X','TCP_Y','TCP_Z',
         'Sensor_P_1_1','Sensor_P_1_2','Sensor_P_2_1','Sensor_P_2_2',
         'Sensor_P_3_1','Sensor_P_3_2','Sensor_P_4_1','Sensor_P_4_2',
         'Sensor_P_5_1','Sensor_P_5_2','Sensor_P_6_1','Sensor_P_6_2',
         'Sensor_S']
Total_dataset=A.read_Data_pandas(Columns,New_Columns)
Total_dataset

1st Folder name :  first_0
Contents in the folder : ['metadata.yaml', 'first_0.db3'] 


1 Parameters ['first'] 

2 Parameters ['0'] 



,Force_X,Force_Y,Force_Z,Torque_X,Torque_Y,Torque_Z,Input_P_1,Input_P_2,Flag,TCP_X,...,Sensor_P_3_2,Sensor_P_4_1,Sensor_P_4_2,Sensor_P_5_1,Sensor_P_5_2,Sensor_P_6_1,Sensor_P_6_2,Sensor_S,obj_p1,obj_p2
0,0.000794,-0.205805,0.490874,-0.005637,0.000218,-0.000148,0.0,0.0,False,-0.034962,...,773.700012,747.299988,777.799988,1036.500000,1045.699951,527.400024,798.799988,6694.399902,first,0
1,0.000794,-0.205805,0.490874,-0.005637,0.000218,-0.000148,0.0,0.0,False,-0.034962,...,773.700012,747.299988,777.799988,1036.500000,1045.699951,527.400024,798.799988,6691.500000,first,0
2,0.000794,-0.205805,0.490874,-0.005637,0.000218,-0.000148,0.0,0.0,False,-0.034962,...,773.700012,747.299988,777.799988,1036.500000,1045.699951,527.400024,798.799988,6691.500000,first,0
3,0.000794,-0.205805,0.490874,-0.005637,0.000218,-0.000148,0.0,0.0,False,-0.034977,...,773.700012,747.299988,777.799988,1036.500000,1045.699951,527.400024,798.799988,6691.500000,first,0
4,0.000794,-0.205805,0.490874,-0.005637,0.000218,-0.000148,0.0,0.0,False,-0.034977,...,773.700012,747.299988,777.799988,1036.500000,1045.699951,527.400024,798.799988,6691.500000,first,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,0.001068,-0.207219,0.503291,-0.005527,0.000286,-0.000279,0.0,0.0,False,-0.035039,...,774.200012,744.799988,774.299988,1034.199951,1041.600098,523.000000,797.799988,6678.199707,first,0
20596,0.001068,-0.207219,0.503291,-0.005527,0.000286,-0.000279,0.0,0.0,False,-0.035039,...,774.200012,744.799988,774.299988,1034.199951,1041.600098,523.000000,797.799988,6678.199707,first,0
20597,0.001068,-0.207219,0.503291,-0.005527,0.000286,-0.000279,0.0,0.0,False,-0.035039,...,774.200012,744.799988,774.299988,1034.199951,1041.600098,523.099976,798.500000,6678.199707,first,0
20598,0.001068,-0.207219,0.503291,-0.005527,0.000286,-0.000279,0.0,0.0,False,-0.035028,...,774.200012,744.799988,774.299988,1034.199951,1041.600098,523.099976,798.500000,6678.199707,first,0


### 3-1. FIltering

In [ ]:
## Data filtering
# https://builtin.com/data-science/pandas-filter
Filtered_dataset=Total_dataset.query('Flag==True')
Filtered_dataset


In [ ]:
Total_dataset[S_Sensor_col].plot()

In [ ]:
Fol_col=Total_dataset.columns[:3]
Input_P_col=Total_dataset.columns[6:8]
Flag_col=Total_dataset.columns[8]
P_Sensor_col=Total_dataset.columns[9:21]
S_Sensor_col=Total_dataset.columns[21]


fig, axes = plt.subplots(nrows=4, ncols=1)

Total_dataset[Fol_col].plot(ax=axes[0])
Total_dataset[Input_P_col].plot(ax=axes[1])
Total_dataset[S_Sensor_col].plot(ax=axes[2])
Total_dataset[P_Sensor_col].plot(ax=axes[3])



In [6]:
Total_dataset.to_pickle("./pickle/tcp_trial.pkl") 

### 3-2 Combination of the data

In [ ]:
Total_col=[*Columns,*New_Columns]
for i in range(len(Total_col)):
    if i==0:
        print(f'{"Total column" :^120}')
        print('')
    print(f'{i:<} {Total_col[i] : <30}   ',end='')
    if i%3 ==0 and i!=0:
        print('')
        

Desired_combination_COL=[*Total_col[0:3],*Total_col[-9:-5]] ####################### chose the combination of the col
print('')
for i in range(len(Desired_combination_COL)):
    if i==0:
        print(f'{"Desired_combination_COL" :^120}')
        print('')
    print(f'{i:<} {Desired_combination_COL[i] : <30}   ',end='')
    if i%3 ==0 and i!=0:
        print('')


Desired_combination_COL
Combi_df=Filtered_dataset.filter(items=Desired_combination_COL)

### 3. Save the file into pickle format

In [5]:
save_mode= True

if save_mode== True:
    Combi_df.to_pickle("./pickle/tcp_test_1.pkl") 

NameError: name 'Combi_df' is not defined

# 3.Start

### 3-1. Data loading and preparation
1) Read pickle
2) Divide Input and output && Train and test set
3) To Tensor
4) Normalization
5) Batch size

In [2]:
Combi_df = pd.read_pickle("./pickle/test_1.pkl")  


# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
#!nvidia-smi

# Hot encoding needed
# Scailing!
# Validation dataset?

Y_t=np.array(Combi_df.filter(items=Desired_combination_COL[0:3]))
X_t=np.array(Combi_df.filter(items=Desired_combination_COL[3:]))

# Scaler
X_scaler = sklearn.preprocessing.MinMaxScaler()
Y_scaler = sklearn.preprocessing.MinMaxScaler()

X=torch.FloatTensor(X_scaler.fit_transform(X_t))
Y=torch.FloatTensor(Y_scaler.fit_transform(Y_t))



X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible


train_dataset=DataLoader(data_loader.Data(X_train,Y_train),batch_size=BATCH_SIZE)
test_dataset=DataLoader(data_loader.Data(X_test,Y_test),batch_size=BATCH_SIZE)
Input_dim=len(X_train[0])
Output_dim=len(Y_train[0])

print(f'{"Input dim ":>20}{Input_dim}{"||":^25}{"Output dim ":>20}{Output_dim}')
print(f'Input Train: {len(X_train)},Test: {len(X_test)}   ||   Output Train: {len(Y_train)}, Test: {len(Y_test)}')

NameError: name 'np' is not defined

### 3-2. Set hypermeter
1. Epochs
2. Batch size
3. Learning rate
4. Device 
5. Activation function
6. Loss function
7. Optimizer

In [ ]:
Input_dim

In [ ]:

# Model

model = model_builder.LSTMModel1(
    input_dim = Input_dim,
    hidden_dim=HIDDEN_UNITS,
    layer_dim=2,
    output_dim=Output_dim,
    dropout_prob=0.7)


# Set loss and optimizer
loss_fn = torch.nn.MSELoss() ## change
optimizer = torch.optim.Adam(model.parameters(), ## change
                             lr=LEARNING_RATE)


### 3-3. Model


In [ ]:

#torchinfo.summary(model,(5,5,5))
start_time = timer()
results=engine.train(model=model,
             train_dataloader=train_dataset,
             test_dataloader=test_dataset,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")
utils.plot_loss_curves([results])

### 3-4 Compare the results and save

In [ ]:

results
